In [1]:
# Purpose: Import necessary libraries for data manipulation, API calls, and text processing.

# Standard library imports
import os
import random
import re

# Third-party library imports
import pandas as pd
from tqdm import tqdm
from google import genai
from langchain_community.llms import Ollama
from langchain.schema import AIMessage, HumanMessage

In [2]:
# Purpose: Load the Gemini API key from an environment variable.

# Load the API key from an environment variable
YOUR_API_KEY = os.environ.get("GEMINI_API_KEY")

if YOUR_API_KEY is None:
    raise ValueError("GEMINI_API_KEY environment variable not set.")

In [3]:
# Purpose: Initialize the Gemini API client and define the call_gemini function.

from google import genai
from google.genai import types

# Initialize Google Gemini API client
client = genai.Client(api_key=YOUR_API_KEY)


def call_gemini(prompt_text, temperature=0.7, max_output_tokens=500, top_p=0.95, top_k=40):
    """
    Call the Gemini API to generate content with configurable parameters.

    Args:
        prompt_text (str): The prompt text to send to the Gemini API.
        temperature (float, optional): Controls the randomness of the output. Defaults to 0.7.
        max_output_tokens (int, optional): The maximum number of tokens in the output. Defaults to 500.
        top_p (float, optional): Nucleus sampling parameter. Defaults to 0.95.
        top_k (int, optional): Top-K sampling parameter. Defaults to 40.

    Returns:
        str: The generated text response from the Gemini API.
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash-lite",  # Use the desired model
        contents=prompt_text,
        config=types.GenerateContentConfig(
            temperature=temperature,          # Controls randomness
            max_output_tokens=max_output_tokens,  # Maximum tokens in the output
            topP=top_p,                       # Nucleus sampling
            topK=top_k                        # Top-K sampling
        )
    )
    return response.text  # Extract the text response

In [4]:
# Purpose: Load and preprocess the existing spam dataset.

import os

# Constants
DATASET_PATH = os.path.join('..', '..', '..', 'Data', 'Farsi', '1. Main', 'farsi_sms.csv')
TEXT_COLUMN = 'Text'
CLASS_COLUMN = 'Class'
LABEL_COLUMN = 'label'
MESSAGE_COLUMN = 'message'

# Load and preprocess spam dataset
spam_data = pd.read_csv(DATASET_PATH, encoding='UTF-8')
spam_data = spam_data[[TEXT_COLUMN, CLASS_COLUMN]].rename(columns={CLASS_COLUMN: LABEL_COLUMN, TEXT_COLUMN: MESSAGE_COLUMN})


def load_existing_data():
    """
    Loads spam and ham messages from the dataset.

    Returns:
        tuple: A tuple containing two lists:
            - ham_samples (list): A list of ham messages.
            - spam_samples (list): A list of spam messages.
    """
    try:
        ham_samples = spam_data[spam_data[LABEL_COLUMN] == "ham"][MESSAGE_COLUMN].tolist()
        spam_samples = spam_data[spam_data[LABEL_COLUMN] == "spam"][MESSAGE_COLUMN].tolist()
        return ham_samples, spam_samples
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return [], []

In [ ]:
# Purpose: Generate synthetic SMS messages in batches using the Gemini API.

def generate_sms_batch(category: str = "ham", num_samples: int = 10, messages_per_request: int = 5, temperature: float = 0.95):
    """
    Generates synthetic SMS messages in batches using the previous prompt template.
    Note: Text processing is removed and will be applied later on the generated dataset.

    Args:
        category (str, optional): The category of SMS messages ("ham" or "spam"). Defaults to "ham".
        num_samples (int, optional): Total number of messages to generate. Defaults to 10.
        messages_per_request (int, optional): Number of messages per API request. Defaults to 5.
        temperature (float, optional): Temperature parameter for text generation. Defaults to 0.95.

    Returns:
        List[str]: List of generated SMS messages.
    """
    ham_examples, spam_examples = load_existing_data()
    messages = []
    recent_messages = set()  # To prevent immediate repetition

    for i in range(0, num_samples, messages_per_request):
        example = random.choice(spam_examples if category == "spam" else ham_examples)
        show_example = (random.random() < 0.5)
        example_text = f"نمونه‌ای از پیام‌های {category}: \"{example}\"" if show_example else "پیام‌ها باید طبیعی و مرتبط باشند."

        base_prompt = (
            f"شما یک دستیار هوش مصنوعی هستید که باید **پنج پیامک واقعی و طبیعی ({category}) به زبان فارسی** تولید کنید.\n"
            "فقط پیامک‌ها را تولید کنید و هیچ متن اضافی، شماره‌گذاری یا توضیحی اضافه نکنید.\n"
            f"{example_text}\n\n### دستورالعمل‌ها:\n"
            "1. هر پیام را **کوتاه** (کمتر از 45 کلمه) و **طبیعی** نگه دارید.\n"
            "2. از زبان **محاوره‌ای و متنوع** استفاده کنید.\n"
            "3. پیام‌ها را **به صورت جداگانه** در خطوط جداگانه بنویسید.\n"
            "4. متن اضافی، شماره‌گذاری یا توضیحی اضافه نکنید.\n"
        )

        if category == "ham":
            prompt = (
                base_prompt +
                "5. پیام‌ها می‌توانند شامل **دعوت‌ها، هماهنگی‌ها، شوخی‌ها، یادآوری‌ها، بحث‌های کوتاه روزمره** باشند.\n"
                "6. پیام‌ها باید حس **یک مکالمه واقعی** را منتقل کنند، نه صرفاً جمله‌های کوتاه بی‌معنی.\n"
                "7. از جملات معمولی و متنوع استفاده کنید، مانند صحبت درباره‌ی **اتفاقات روزمره، ورزش و فعالیت‌های بدنی، غذا، سفر و تعطیلات، فیلم و سریال، دوستان و خانواده، تکنولوژی و گجت‌ها، کار و دانشگاه**.\n"
                "\nاکنون پنج پیامک طبیعی تولید کنید."
            )
        elif category == "spam":
            prompt = (
                base_prompt +
                "5. پیام‌ها باید شامل **تبلیغات هوشمندانه، پیشنهادات تجاری، قرعه‌کشی‌ها، هشدارهای بانکی مشکوک، یا وام‌های مشکوک** باشند.\n"
                "6. از کلیشه‌های تکراری مثل \"برنده شدید!\" بیش از حد استفاده نکنید؛ سعی کنید **متن‌های متنوع‌تر و فریبنده‌تر** تولید کنید.\n"
                "7. پیام‌ها باید **حس فوریت** ایجاد کنند ولی بیش از حد جعلی و کلیشه‌ای نباشند.\n"
                "8. از نمادهای غیرضروری (🔥💰🎁) کمتر استفاده کنید.\n"
                "\nاکنون پنج پیامک تولید کنید."
            )
        else:
            raise ValueError("Invalid category. Choose 'ham' or 'spam'.")

        # Call the Gemini API using the constructed prompt
        raw_response = call_gemini(prompt, temperature=temperature)
        new_messages = raw_response.strip().split("\n")

        # Append the raw messages without processing
        for msg in new_messages:
            stripped_msg = msg.strip()
            if stripped_msg and stripped_msg not in recent_messages:
                messages.append(stripped_msg)
                recent_messages.add(stripped_msg)

    return messages

In [6]:
# Purpose: Generate a synthetic SMS dataset and save it incrementally as a CSV file.

import os
import pandas as pd
from tqdm import tqdm
from typing import List

def generate_dataset(num_ham: int = 500, num_spam: int = 500, output_file: str = "../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv", batch_size: int = 10) -> None:
    """
    Generates a synthetic SMS dataset and saves it incrementally as a CSV file.

    Args:
        num_ham (int, optional): The number of ham messages to generate. Defaults to 500.
        num_spam (int, optional): The number of spam messages to generate. Defaults to 500.
        output_file (str, optional): The path to the output CSV file. Defaults to "../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv".
        batch_size (int, optional): The number of messages to generate in each batch. Defaults to 10.
    """

    # Initialize or load existing dataset
    if os.path.exists(output_file):
        try:
            data = pd.read_csv(output_file, encoding='UTF-8')
            print(f"Loaded existing dataset with {len(data)} samples.")
        except Exception as e:
            print(f"Error loading existing dataset: {e}. Starting with an empty dataset.")
            data = pd.DataFrame(columns=["message", "label"])
    else:
        data = pd.DataFrame(columns=["message", "label"])

    # Track progress
    ham_generated = len(data[data["label"] == "ham"])
    spam_generated = len(data[data["label"] == "spam"])

    # Generate ham messages
    with tqdm(total=num_ham, initial=ham_generated, desc="Generating Ham Messages", unit="msg") as ham_progress:
        while ham_generated < num_ham:
            ham_batch = generate_sms_batch(category="ham", num_samples=min(batch_size, num_ham - ham_generated))
            new_ham_data = pd.DataFrame({"message": ham_batch, "label": ["ham"] * len(ham_batch)})
            data = pd.concat([data, new_ham_data], ignore_index=True)
            ham_generated += len(ham_batch)
            ham_progress.update(len(ham_batch))

    # Generate spam messages
    with tqdm(total=num_spam, initial=spam_generated, desc="Generating Spam Messages", unit="msg") as spam_progress:
        while spam_generated < num_spam:
            spam_batch = generate_sms_batch(category="spam", num_samples=min(batch_size, num_spam - spam_generated))
            new_spam_data = pd.DataFrame({"message": spam_batch, "label": ["spam"] * len(spam_batch)})
            data = pd.concat([data, new_spam_data], ignore_index=True)
            spam_generated += len(spam_batch)
            spam_progress.update(len(spam_batch))

    # Shuffle the dataset before saving
    data = data.sample(frac=1).reset_index(drop=True)

    # Save the dataset to a CSV file
    try:
        data.to_csv(output_file, index=False, encoding='UTF-8')
        print(f"Dataset generation completed. Saved to {output_file}. Total samples: {len(data)}")
    except Exception as e:
        print(f"Error saving dataset to {output_file}: {e}")

In [7]:
# Purpose: Generate the synthetic SMS dataset.

# Define the output file path
output_file = "../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv"

# Check if the output directory exists
output_dir = os.path.dirname(output_file)
if not os.path.exists(output_dir):
    try:
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")
    except Exception as e:
        print(f"Error creating output directory: {e}. Please ensure the path is valid.")
        # Optionally, you can exit the script here if directory creation fails
        # return

# Generate dataset
generate_dataset(num_ham=1000, num_spam=1000, output_file=output_file)

Loaded existing dataset with 1900 samples.


Generating Ham Messages: 1005msg [00:18,  2.97msg/s]                        
Generating Spam Messages: 100%|██████████| 1000/1000 [00:25<00:00,  1.95msg/s]

Dataset generation completed. Saved to ../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv. Total samples: 2005


In [8]:
# Purpose: Read the generated synthetic SMS dataset.

# Define the output file path
output_file = "../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv"

# Read the synthetic dataset
try:
    synthetic = pd.read_csv(output_file, encoding='UTF-8')
    print(f"Successfully loaded synthetic dataset from {output_file} with {len(synthetic)} rows.")
    print(synthetic)  # Display the first few rows
except FileNotFoundError:
    print(f"Error: The file {output_file} was not found. Please ensure the dataset has been generated.")
except Exception as e:
    print(f"An error occurred while reading the dataset: {e}")

Successfully loaded synthetic dataset from ../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv with 2005 rows.
                                                message label
0     "سلام، یادم رفت بهت بگم، امتحان فیزیک رو انداخ...   ham
1     مهلت استفاده از تسهیلات بانکی شما رو به پایانه...  spam
2       کلاس ساعت ۱۰ تشکیل میشه، یادم رفت بهت بگم! 🤦‍♀️   ham
3                  وایسا دم در، زود بیا! منم دارم میام.   ham
4     امروز تولد مامانمه! چی‌کار کنیم سورپرایزش کنیم...   ham
...                                                 ...   ...
2000                               اوکی، این پنج پیامک:   ham
2001  فروش ویژه پاییزی! تخفیف 70% در تمامی محصولات! ...  spam
2002                   سلام، شام چی داریم امشب؟ گرسنمه!   ham
2003                   باید برم باشگاه. بعدا زنگ میزنم.   ham
2004  1.  سلام! امشب فیلم چی ببینیم؟ یه چیزی باشه که...   ham

[2005 rows x 2 columns]


In [9]:
# Purpose: Extract the SMS messages from the synthetic dataset into a list.

# Extract messages from the synthetic dataset
if not synthetic.empty:
    messages = synthetic["message"].tolist()
    print(f"Extracted {len(messages)} messages from the synthetic dataset.")
else:
    print("The synthetic dataset is empty. No messages were extracted.")
    messages = []

Extracted 2005 messages from the synthetic dataset.


.

.

.

.

.


### lets define some function  to prcec the output samples

In [ ]:
# Purpose: Consolidate all text processing functions into a single cell.

import re
from typing import List, Tuple
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def clean_message(raw_text: str) -> str:
    """
    Removes the AI's thought process enclosed in <think>...</think> tags.

    Args:
        raw_text (str): The input text that may contain <think>...</think> tags.

    Returns:
        str: The cleaned text with the <think>...</think> tags removed.
    """
    return re.sub(r"<think>.*?</think>", "", raw_text, flags=re.DOTALL).strip()

def clean_numbered_messages(message: str) -> str:
    """
    Remove numbers and dots at the beginning of a message.

    Args:
        message (str): The input message containing numbers and dots.

    Returns:
        str: The cleaned message without leading numbers and dots.
    """
    return re.sub(r"^\d+\.\s*", "", message).strip()

def remove_numbers_from_messages(message: str) -> str:
    """
    Remove numbers and associated punctuation at the beginning of a message.

    Args:
        message (str): The input message containing numbers and punctuation.

    Returns:
        str: The cleaned message without leading numbers and punctuation.
    """
    return re.sub(r"^\s*\d+[.\)\-:\s]?\s*", "", message).strip()

def replace_placeholders(message: str) -> str:
    """
    Replace placeholders in the generated message with realistic values.

    Args:
        message (str): The input message containing placeholders.

    Returns:
        str: The message with placeholders replaced by realistic values.
    """
    import random

    fake_phone = f"0912{random.randint(1000000, 9999999):07d}"
    fake_account_number = f"{random.randint(1000000000000000, 9999999999999999)}"
    fake_url = f"www.example{random.randint(1, 100)}.com"
    fake_website_name = f"example{random.randint(1, 100)}.com"
    fake_store_name = f"فروشگاه {random.choice(['ایران', 'تهران', 'آنلاین'])}"
    fake_telegram_link = f"t.me/fakechannel{random.randint(1, 10)}"
    fake_code = f"{random.randint(1000, 9999)}"

    replacements = {
        "[لینک جعلی]": fake_url,
        "[لینک فیشینگ]": fake_url,
        "[لینک]": fake_url,
        "[شماره حساب]": fake_account_number,
        "[نام وبسایت]": fake_website_name,
        "[لینک فروشگاه]": fake_url,
        "[لینک وبسایت]": fake_url,
        "[لینک سایت]": fake_url,
        "[نام موسسه]": fake_store_name,
        "[آدرس سایت]": fake_url,
        "[لینک ثبت نام]": fake_url,
        "[لینک تلگرام]": fake_telegram_link,
        "[شماره تماس جعلی]": fake_phone,
        "[شماره جعلی]": fake_phone,
        "[شماره تلفن]": fake_phone,
        "[آدرس وب‌سایت]": fake_url,
        "[آدرس وبسایت]": fake_url,
        "[لینک‌کوتاه]": fake_url,
        "[آدرس]": fake_url,
        "[لینک_وام]": fake_url,
        "[لینک وام]": fake_url,
        "[لینک خرید]": fake_url,
        "[کد فعال‌سازی]": fake_code,
        "[لینک تبلیغاتی]": fake_url,
        "[لینک_تبلیغاتی]": fake_url,
        "[لینک کوتاه شده]": fake_url,
        "[لینک فیک]": fake_url,
        "[لینک ثبت‌نام]": fake_url,
        "[شماره_تماس_جعلی]": fake_phone,
        "[لینک کلاهبرداری]": fake_url,
        "[لینک‌جعلی]": fake_url,
        "[لینک‌تقلبی]": fake_url,
        "[لینک نامعتبر]": fake_url,
        "[لینک دروغین]": fake_url,
        "[کد]": fake_code,
        "[کد تخفیف]": fake_code,
        "[کد تایید]": fake_code,
        "[کد اعتبار سنجی]": fake_code,
        "[کد فعالسازی]": fake_code,
        "[شماره تماس]": fake_phone,
        "[شماره]": fake_phone,
        "[نام فروشگاه]": fake_store_name,
        "[آدرس/شماره تماس]": fake_store_name,
    }

    for placeholder, replacement in replacements.items():
        message = message.replace(placeholder, replacement)

    return message

def identify_similar_messages(messages: List[str], threshold: float = 0.8) -> Tuple[List[str], List[Tuple[str, str, float]], List[int]]:
    """
    Identifies and removes pairs of messages that are too similar to each other based on cosine similarity.

    Args:
        messages (List[str]): A list of SMS messages.
        threshold (float, optional): The similarity threshold. Defaults to 0.8.

    Returns:
        Tuple[List[str], List[Tuple[str, str, float]], List[int]]: A tuple containing:
            - A list of unique messages.
            - A list of similar message pairs with their similarity scores.
            - A list of indices of the unique messages in the original list.
    """
    vectorizer = TfidfVectorizer().fit_transform(messages)
    similarity_matrix = cosine_similarity(vectorizer)
    similar_pairs = []
    unique_messages = []
    unique_indices = []  # Keep track of the indices of unique messages

    for i, message in enumerate(messages):
        # Remove numbers from the message
        message = remove_numbers_from_messages(message)

        is_unique = True
        for j, unique_index in enumerate(unique_indices):
            # Calculate cosine similarity between the current message and the unique message
            similarity = similarity_matrix[i, unique_index]

            # If the similarity is above the threshold, mark the current message as not unique
            if similarity > threshold:
                similar_pairs.append((messages[i], messages[unique_index], similarity))
                is_unique = False
                break

        # If the message is unique, add it to the list of unique messages and its index to the list of unique indices
        if is_unique:
            unique_messages.append(messages[i])
            unique_indices.append(i)

    return unique_messages, similar_pairs, unique_indices

.



.



.



.



.


#### now let's execute them on the data

In [ ]:
# Revised cell: Process synthetic SMS text with progress bars and detailed deletion logs

import pandas as pd
import numpy as np
from tqdm import tqdm

def process_synthetic_text_with_progress(df):
    """
    Applies text processing functions to the 'message' column with progress bars.
    Handles missing values, identifies similar messages, and saves the cleaned dataset.

    Args:
        df (pd.DataFrame): DataFrame containing the synthetic SMS messages.

    Returns:
        pd.DataFrame: DataFrame with processed messages.
    """
    # Step 1: Handle missing values
    print("Step 1: Handling missing values...")
    with tqdm(total=len(df), desc="Filling NaN values", unit="msg") as pbar:
        df["message"] = df["message"].fillna("")
        pbar.update(len(df))

    # Step 2: Apply cleaning functions
    print("Step 2: Cleaning messages...")
    with tqdm(total=len(df), desc="Cleaning messages", unit="msg") as pbar:
        df["message"] = df["message"].apply(
            lambda x: replace_placeholders(
                remove_numbers_from_messages(
                    clean_message(str(x))
                )
            )
        )
        pbar.update(len(df))

    # Step 3: Identify and remove similar messages
    print("Step 3: Identifying and removing similar messages...")
    messages = df["message"].tolist()
    unique_messages, similar_pairs, unique_indices = identify_similar_messages(messages, threshold=0.75)

    # Log similar messages that will be removed
    if similar_pairs:
        print("Similar messages identified for removal:")
        for msg1, msg2, similarity in similar_pairs:
            print(f"Message 1: {msg1}")
            print(f"Message 2: {msg2}")
            print(f"Similarity: {similarity:.4f}")
            print("-" * 50)

    # Progress bar for removing similar messages
    with tqdm(total=len(messages) - len(unique_messages), desc="Removing similar messages", unit="msg") as pbar:
        df = df.iloc[unique_indices].copy()
        pbar.update(len(messages) - len(unique_messages))

    # Reset index after processing
    df.reset_index(drop=True, inplace=True)
    return df


# Define the path to the synthetic dataset CSV file
input_file = "../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv"
output_file = "../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv"  

# Load the synthetic dataset, process the text, and save it to a new file
try:
    synthetic_df = pd.read_csv(input_file, encoding="UTF-8")
    print(f"Loaded synthetic dataset with {len(synthetic_df)} rows.")
    print("Sample before processing:")
    print(synthetic_df.head())

    processed_df = process_synthetic_text_with_progress(synthetic_df.copy())

    # Save the processed dataset to a NEW CSV file
    processed_df.to_csv(output_file, index=False, encoding="UTF-8")
    print(f"Synthetic dataset text processed and saved successfully to {output_file}")

except FileNotFoundError:
    print(f"Error: Input file not found: {input_file}")
except Exception as e:
    print(f"Error processing synthetic dataset: {e}")

Loaded synthetic dataset with 1972 rows.
Sample before processing:
                                             message label
0  "سلام، یادم رفت بهت بگم، امتحان فیزیک رو انداخ...   ham
1  مهلت استفاده از تسهیلات بانکی شما رو به پایانه...  spam
2    کلاس ساعت ۱۰ تشکیل میشه، یادم رفت بهت بگم! 🤦‍♀️   ham
3               وایسا دم در، زود بیا! منم دارم میام.   ham
4  امروز تولد مامانمه! چی‌کار کنیم سورپرایزش کنیم...   ham
Step 1: Handling missing values...


Filling NaN values: 100%|██████████| 1972/1972 [00:00<?, ?msg/s]


Step 2: Cleaning messages...


Cleaning messages: 100%|██████████| 1972/1972 [00:00<00:00, 182421.37msg/s]

Step 3: Identifying and removing similar messages...


Similar messages identified for removal:
Message 1: وام فوری تا 50 میلیون تومان بدون ضامن! فقط با یک تماس! فرصت رو از دست ندهید! با شماره 09128470924 تماس بگیرید.
Message 2: وام فوری تا 50 میلیون تومان بدون ضامن! فقط با یک تماس! فرصت رو از دست ندید. برای اطلاعات بیشتر با شماره 021... تماس بگیرید.
Similarity: 0.7547
--------------------------------------------------
Message 1: شارژ رایگان ایرانسل! کد 12345 رو به 8000 بفرست تا اعتبار بگیری، فقط تا امشب فرصت داری!
Message 2: شارژ رایگان ایرانسل! کد 12345 رو به 8888 بفرست تا اعتبار بگیری! فرصت محدوده!
Similarity: 0.7556
--------------------------------------------------
Message 1: فروش ویژه آپارتمان‌های لوکس در شمال! بازدید امروز، تخفیف ویژه! تماس: 0935...
Message 2: فروش ویژه آپارتمان‌های لوکس با شرایط استثنایی! بازدید رایگان، همین امروز تماس بگیر: 0935...
Similarity: 0.7622
--------------------------------------------------
Message 1: فیلم جدید فلانی رو دیدی؟ خیلی باحال بود، حتماً ببین!
Message 2: فیلم دیدی دیشب؟ خیلی باحال بود، حتما ببی

Removing similar messages: 100%|██████████| 46/46 [00:00<00:00, 34663.67msg/s]

Synthetic dataset text processed and saved successfully to ../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv


In [16]:
synthetic = pd.read_csv("../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv", encoding='UTF-8')
synthetic

,message,label
0,"""سلام، یادم رفت بهت بگم، امتحان فیزیک رو انداخ...",ham
1,مهلت استفاده از تسهیلات بانکی شما رو به پایانه...,spam
2,کلاس ساعت ۱۰ تشکیل میشه، یادم رفت بهت بگم! 🤦‍♀️,ham
3,وایسا دم در، زود بیا! منم دارم میام.,ham
4,امروز تولد مامانمه! چی‌کار کنیم سورپرایزش کنیم...,ham
...,...,...
1921,اوکی، این پنج پیامک:,ham
1922,فروش ویژه پاییزی! تخفیف 70% در تمامی محصولات! ...,spam
1923,سلام، شام چی داریم امشب؟ گرسنمه!,ham
1924,باید برم باشگاه. بعدا زنگ میزنم.,ham


In [17]:
for i in range(len(synthetic)):
    print(f"{i + 1}.[{synthetic['label'][i]}]: {synthetic['message'][i]}")

1.[ham]: "سلام، یادم رفت بهت بگم، امتحان فیزیک رو انداختن چهارشنبه! حواست باشه."
2.[spam]: مهلت استفاده از تسهیلات بانکی شما رو به پایانه. برای تمدید یا دریافت وام جدید، همین امروز اقدام کنید. [تماس]
3.[ham]: کلاس ساعت ۱۰ تشکیل میشه، یادم رفت بهت بگم! 🤦‍♀️
4.[ham]: وایسا دم در، زود بیا! منم دارم میام.
5.[ham]: امروز تولد مامانمه! چی‌کار کنیم سورپرایزش کنیم؟ یه ایده داری؟
6.[ham]: راستی، اون فیلمه که معرفی کردی رو دیدم. خیلی باحال بود، مرسی!
7.[spam]: سلام! یه فرصت استثنایی برای سرمایه گذاری! سود تضمینی و بدون ریسک! اطلاعات بیشتر: [لینک نامعلوم]
8.[ham]: هوا افتضاحه! بارون شدید میاد. چتر داری با خودت؟
9.[spam]: اعلامیه مهم بانک... تراکنش مشکوک در حسابتون ثبت شده. برای تایید، سریعا به لینک زیر مراجعه کنید: www.example26.com
10.[ham]: کلاس امروز کنسل شد، ولی استاد گفت تکلیف رو بفرستید.
11.[ham]: راستی، فردا تولد سارا است. کادو چی بگیریم؟
12.[spam]: فروش ویژه عیدانه: مبل راحتی با 70% تخفیف! فقط تا 24 ساعت آینده! برای اطلاعات بیشتر تماس بگیرید: 0912...
13.[spam]: مهلت استفاده از شارژ هدیه ش

In [27]:
input_file = "../../../Data/Farsi/2. Synthetic/synthetic_farsi_sms.csv"

synthetic = pd.read_csv(input_file, encoding='UTF-8')

In [28]:
synthetic

,message,label
0,"""سلام، یادم رفت بهت بگم، امتحان فیزیک رو انداخ...",ham
1,مهلت استفاده از تسهیلات بانکی شما رو به پایانه...,spam
2,کلاس ساعت ۱۰ تشکیل میشه، یادم رفت بهت بگم! 🤦‍♀️,ham
3,وایسا دم در، زود بیا! منم دارم میام.,ham
4,امروز تولد مامانمه! چی‌کار کنیم سورپرایزش کنیم...,ham
...,...,...
1909,کلاس امروز رو فهمیدی؟ من که هنگ کردم!,ham
1910,فروش ویژه پاییزی! تخفیف 70% در تمامی محصولات! ...,spam
1911,سلام، شام چی داریم امشب؟ گرسنمه!,ham
1912,باید برم باشگاه. بعدا زنگ میزنم.,ham
